In [2]:
from datasets import load_dataset

import pandas as pd
ds = load_dataset("fewshot-goes-multilingual/sk_csfd-movie-reviews")
train = ds["train"]
train_csv = train.to_csv("train.csv")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Creating CSV from Arrow format: 100%|██████████| 25/25 [00:01<00:00, 12.80ba/s]


In [4]:
train_df_raw = pd.DataFrame(train)
train_df_raw.tail()

,review_id,rating_str,rating_int,date,comment_language,comment,item_title,item_year,item_kind,item_genres,item_directors,item_screenwriters,item_cast
24995,L9xOccccoc9I3EM79iEOHQ,4/5,4,2004-06-15,sk,"Zábava to bola skvelá, ale chcem vidieť babu,...",Lepší pozdě nežli později,2003,film,"[Komedie, Romantický]",[Nancy Meyers],[Nancy Meyers],"[Jack Nicholson, Diane Keaton, Amanda Peet, Ke..."
24996,aHpKGs512vWPSuMc8wyqNA,2/5,2,2015-09-15,sk,Nudne spracovaný príbeh s dosť naivným scenár...,Speciální jednotka,2011,film,"[Akční, Válečný, Drama]",[Stéphane Rybojad],"[Michael Cooper, Stéphane Rybojad]","[Diane Kruger, Djimon Hounsou, Benoît Magimel,..."
24997,G8m8aj56nJaQzIplLPFtMA,3/5,3,2018-09-02,sk,2 hodiny cakania na skvele rozuzlenie....a ko...,Děsivé dědictví,2018,film,"[Horor, Psychologický, Mysteriózní, Thriller, ...",[Ari Aster],[Ari Aster],"[Toni Collette, Gabriel Byrne, Alex Wolff, Mil..."
24998,Pi5JJKyihbfH8fFeqVEuZA,2/5,2,2012-12-17,sk,"ide o lepsiu zalezitost nez je jednotka, no s...",Halloween II,1981,film,"[Horor, Thriller]",[Rick Rosenthal],"[John Carpenter, Debra Hill]","[Jamie Lee Curtis, Donald Pleasence, Charles C..."
24999,kNvk8UwgzfvXfkWzrhoLqg,0/5,0,2008-09-18,sk,Toto je k smiechu! Všetci čo na tomto robili ...,V.I.P.,(1998–2002),seriál,"[Akční, Komedie, Krimi]","[Farhad Mann, Chuck Bowman, Greg Yaitanes, Nel...","[J.F. Lawton, Rick Suvalle, Hannah Louise Shea...","[Pamela Anderson, Molly Culver, Shaun Baker, N..."


In [ ]:
df = train_df_raw.drop(["review_id", "rating_str", "date", "comment_language", "item_title", "item_year", "item_kind", "item_genres", "item_directors", "item_screenwriters", "item_cast"], axis=1)
df.tail()

df["rating_int"] = df.apply(
    lambda row: 
)

,rating_int,comment
24995,4,"Zábava to bola skvelá, ale chcem vidieť babu,..."
24996,2,Nudne spracovaný príbeh s dosť naivným scenár...
24997,3,2 hodiny cakania na skvele rozuzlenie....a ko...
24998,2,"ide o lepsiu zalezitost nez je jednotka, no s..."
24999,0,Toto je k smiechu! Všetci čo na tomto robili ...
